# Transformers from Scratch
This is a workthrough of [the nice derivation found here.](https://e2eml.school/transformers.html#table_lookup)

# Setup

## Imports

In [1]:
import numpy as np
import pandas as pd

# Contents

## One-hot encoding

* Words -> numerical representation.

Consider "find my files":

\begin{matrix}
\rm{files} & \rm{find} & \rm{my}
\end{matrix}

\begin{pmatrix}
0 & 1 & 0 \\
0 & 0 & 1 \\
1 & 0 & 0
\end{pmatrix}


In [6]:
# Pandas one-hot encoding
phrase = pd.Series(['find', 'my', 'files'])
encoded_phrase = pd.get_dummies(phrase) # one-hot encoding
encoded_phrase.astype(int)

,files,find,my
0,0,1,0
1,0,0,1
2,1,0,0
